In [1]:
import h3
import json
import pandas as pd
import geopandas as gpd
from shapely.geometry import Polygon

In [2]:
# read geojson file
f = open('../gangnamGu.geojson')
data = json.load(f)
f.close()

In [3]:
# because h3 get coordinates as reversed order of geojson file. I have to reverse the order of coordinates prior to get hexagons
new_coords = [[coord[1],coord[0]] for coord in data['features'][0]['geometry']['coordinates'][0]]

In [4]:
# generate a new geojson file with the reversed coordinates
new_geojson = {'type':'Polygon', 'coordinates':[new_coords]}

In [5]:
# get indices of h3 hexagons from a new geojson 
hexagons = list(h3.polyfill(new_geojson, res=9))
print(hexagons)

['8930e1caa2bffff', '8930e1c8487ffff', '8930e1c8c33ffff', '8930e1c818bffff', '8930e1c881bffff', '8930e1ca433ffff', '8930e1c81a7ffff', '8930e1caa23ffff', '8930e1ca1dbffff', '8930e1c851bffff', '8930e1ca5cbffff', '8930e1ca113ffff', '8930e1c80bbffff', '8930e1c884fffff', '8930e1c8c87ffff', '8930e1ca12fffff', '8930e1c124bffff', '8930e1ca533ffff', '8930e1ca123ffff', '8930e1c8c43ffff', '8930e1c8533ffff', '8930e1c8017ffff', '8930e1c850fffff', '8930e1c81abffff', '8930e1c812fffff', '8930e1c880bffff', '8930e1c8547ffff', '8930e1ca467ffff', '8930e1c8553ffff', '8930e1c8c0fffff', '8930e1c8ccfffff', '8930e1c884bffff', '8930e1ca343ffff', '8930e1ca5dbffff', '8930e1ca23bffff', '8930e1caa77ffff', '8930e1ca523ffff', '8930e1caadbffff', '8930e1c8c7bffff', '8930e1c8ec7ffff', '8930e1c8c37ffff', '8930e1ca2bbffff', '8930e1ca083ffff', '8930e1c8e37ffff', '8930e1c8513ffff', '8930e1c88d3ffff', '8930e1c8e97ffff', '8930e1ca18fffff', '8930e1c85a3ffff', '8930e1ca31bffff', '8930e1c8e8fffff', '8930e1ca1b7ffff', '8930e1cae9

In [6]:
# get centerpoints from the indices
hexagons_centerpoints = [[index, h3.h3_to_geo(index)[1],h3.h3_to_geo(index)[0]] for index in hexagons]
print(hexagons_centerpoints)

[['8930e1caa2bffff', 127.09088007674602, 37.49385852398891], ['8930e1c8487ffff', 127.04323721756204, 37.48843298276747], ['8930e1c8c33ffff', 127.10597856855853, 37.48624478489637], ['8930e1c818bffff', 127.076602325662, 37.47995440527451], ['8930e1c881bffff', 127.11517040275075, 37.47080567619576], ['8930e1ca433ffff', 127.01764129459687, 37.52730942790988], ['8930e1c81a7ffff', 127.08745456709589, 37.48429692213922], ['8930e1caa23ffff', 127.0911174153499, 37.49668392947605], ['8930e1ca1dbffff', 127.0379236248607, 37.51906416778404], ['8930e1c851bffff', 127.0615100570407, 37.48756306031005], ['8930e1ca5cbffff', 127.02306301779775, 37.52948492626227], ['8930e1ca113ffff', 127.04605957348535, 37.522325918955566], ['8930e1c80bbffff', 127.06304207283702, 37.47452617494918], ['8930e1c884fffff', 127.11716985031731, 37.463413405365806], ['8930e1c8c87ffff', 127.09854806920563, 37.49146490035668], ['8930e1ca12fffff', 127.05891613592362, 37.51928716727716], ['8930e1c124bffff', 127.04676563699253, 37

In [7]:
hexagons_polygons =[Polygon([(coords[1], coords[0]) for coords in h3.h3_to_geo_boundary(index)]) for index in hexagons]
print(hexagons_polygons[:5])

[<shapely.geometry.polygon.Polygon object at 0x0000015951E72910>, <shapely.geometry.polygon.Polygon object at 0x0000015951E727C0>, <shapely.geometry.polygon.Polygon object at 0x000001594F4D5AF0>, <shapely.geometry.polygon.Polygon object at 0x000001594F4D5760>, <shapely.geometry.polygon.Polygon object at 0x000001594F4D5430>]


In [8]:
# create a dataframe with the indices and centerpoints
df_hexagons = pd.DataFrame(hexagons_centerpoints, columns =['index', 'x','y'])

In [9]:
df_hexagons.head()

,index,x,y
0,8930e1caa2bffff,127.090880,37.493859
1,8930e1c8487ffff,127.043237,37.488433
2,8930e1c8c33ffff,127.105979,37.486245
3,8930e1c818bffff,127.076602,37.479954
4,8930e1c881bffff,127.115170,37.470806


In [10]:
# convert the dataframe into a geodataframe with geometries
gdf_hexagons = gpd.GeoDataFrame(df_hexagons, geometry = hexagons_polygons)

In [11]:
gdf_hexagons.head()

,index,x,y,geometry
0,8930e1caa2bffff,127.090880,37.493859,"POLYGON ((127.09261 37.49364, 127.09186 37.495..."
1,8930e1c8487ffff,127.043237,37.488433,"POLYGON ((127.04497 37.48822, 127.04422 37.489..."
2,8930e1c8c33ffff,127.105979,37.486245,"POLYGON ((127.10771 37.48603, 127.10696 37.487..."
3,8930e1c818bffff,127.076602,37.479954,"POLYGON ((127.07833 37.47974, 127.07759 37.481..."
4,8930e1c881bffff,127.115170,37.470806,"POLYGON ((127.11690 37.47059, 127.11615 37.472..."


In [12]:
# export the geodataframe into a geojson file 
gdf_hexagons.to_file('../hexagon_res_9/hexagons.geojson', encoding='utf8', driver='GeoJSON')